In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/MyDrive/Cs412/project_files

/content/drive/MyDrive/Cs412/project_files


In [ ]:
%ls

 bugs-test.csv                            results/
 bugs-train.csv                           rf_bert_predictions_2.csv
 encoded_texts.pkl                        rf_bert_predictions_3.csv
 encoded_texts_train_test_1.pkl           rf_bert_predictions_4.csv
 encoded_texts_train_test.pkl             rf_bert_predictions_5.csv
 lgbm_p-_sm-.ipynb                        rf_bert_predictions_6.csv
 ngram_vectorizer.pkl                    'submission_02 06 2024.csv'
'predictions_02 06 2024_02_49.csv'        submission_0_51.csv
 predictions.csv                          submission_0_55.csv
 randomforest_predictions_2.csv           submission.csv
 random_forest_smote_model.pkl            tfidf_vectorizer.pkl
 random_forest_smote_model_v1_0.pkl       train_prep.csv
 random_forest_smote_model_v2_tfidf.pkl   Untitled0.ipynb


In [3]:
!pip install imblearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, precision_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import traceback
import numpy as np
from scipy.sparse import isspmatrix, csr_matrix
import pickle

In [4]:
data = pd.read_csv('bugs-train.csv')
# Load test data
test_data = pd.read_csv('bugs-test.csv')
# Severity mapping
severity_mapping = {
    'trivial': 0,
    'enhancement': 1,
    'minor': 2,
    'normal': 3,
    'major': 4,
    'blocker': 5,
    'critical': 6
}
# Map severity values to integers
data['severity'] = data['severity'].map(severity_mapping)


In [ ]:
# Extract features using BERT
X = data['summary'].tolist()
y = data['severity'].values
X_test = test_data['summary'].tolist()

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the text data in batches to manage memory usage
def batch_encode_texts(texts, batch_size=64):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = model_output.last_hidden_state[:, 0, :].numpy()
        embeddings.append(batch_embeddings)
        print( f"Completed batch {i // batch_size + 1}/{len(texts) // batch_size + 1}")
    return np.vstack(embeddings)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
X_encoded = batch_encode_texts(X)

Completed batch 1/2500
Completed batch 2/2500
Completed batch 3/2500
Completed batch 4/2500
Completed batch 5/2500
Completed batch 6/2500
Completed batch 7/2500
Completed batch 8/2500
Completed batch 9/2500
Completed batch 10/2500
Completed batch 11/2500
Completed batch 12/2500
Completed batch 13/2500
Completed batch 14/2500
Completed batch 15/2500
Completed batch 16/2500
Completed batch 17/2500
Completed batch 18/2500
Completed batch 19/2500
Completed batch 20/2500
Completed batch 21/2500
Completed batch 22/2500
Completed batch 23/2500
Completed batch 24/2500
Completed batch 25/2500
Completed batch 26/2500
Completed batch 27/2500
Completed batch 28/2500
Completed batch 29/2500
Completed batch 30/2500
Completed batch 31/2500
Completed batch 32/2500
Completed batch 33/2500
Completed batch 34/2500
Completed batch 35/2500
Completed batch 36/2500
Completed batch 37/2500
Completed batch 38/2500
Completed batch 39/2500
Completed batch 40/2500
Completed batch 41/2500
Completed batch 42/2500
C

In [ ]:
X_test_encoded = batch_encode_texts(X_test)

Completed batch 1/1346
Completed batch 2/1346
Completed batch 3/1346
Completed batch 4/1346
Completed batch 5/1346
Completed batch 6/1346
Completed batch 7/1346
Completed batch 8/1346
Completed batch 9/1346
Completed batch 10/1346
Completed batch 11/1346
Completed batch 12/1346
Completed batch 13/1346
Completed batch 14/1346
Completed batch 15/1346
Completed batch 16/1346
Completed batch 17/1346
Completed batch 18/1346
Completed batch 19/1346
Completed batch 20/1346
Completed batch 21/1346
Completed batch 22/1346
Completed batch 23/1346
Completed batch 24/1346
Completed batch 25/1346
Completed batch 26/1346
Completed batch 27/1346
Completed batch 28/1346
Completed batch 29/1346
Completed batch 30/1346
Completed batch 31/1346
Completed batch 32/1346
Completed batch 33/1346
Completed batch 34/1346
Completed batch 35/1346
Completed batch 36/1346
Completed batch 37/1346
Completed batch 38/1346
Completed batch 39/1346
Completed batch 40/1346
Completed batch 41/1346
Completed batch 42/1346
C

##Imported without test

In [ ]:
# Save encoded texts and labels
with open('encoded_texts.pkl', 'wb') as f:
    pickle.dump((X_encoded, y), f)



In [6]:
# Load encoded texts and labels (for future use)
with open('encoded_texts_512.pkl', 'rb') as f:
    X_encoded, y = pickle.load(f)

## Imported with test

In [ ]:
# Save encoded texts and labels
with open('encoded_texts_train_test_128.pkl', 'wb') as f:
    pickle.dump((X_encoded, y, X_test_encoded), f)


In [5]:

# Load encoded texts and labels (for future use)
with open('encoded_texts_train_test_512.pkl', 'rb') as f:
    X_encoded, y, X_test_encoded = pickle.load(f)

In [8]:
!pip install numpy scikit-learn imbalanced-learn xgboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.0 MB/s eta 0:00:00


In [6]:
from xgboost import XGBClassifier
import xgboost as xgb

In [23]:
!pip install tensorflow

In [24]:
import numpy as np
import traceback
from sklearn.metrics import precision_score, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [36]:

# Custom focal loss function
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_true = tf.keras.backend.clip(y_true, epsilon, 1. - epsilon)
        y_pred = tf.keras.backend.clip(y_pred, epsilon, 1. - epsilon)

        cross_entropy = -y_true * tf.keras.backend.log(y_pred)
        loss = alpha * tf.keras.backend.pow(1 - y_pred, gamma) * cross_entropy
        return tf.keras.backend.mean(tf.keras.backend.sum(loss, axis=1))
    return focal_loss_fixed

# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []

# Standardize the data
scaler = StandardScaler()

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Standardize the features
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # One-hot encode the labels for TensorFlow
        y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)
        y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)

        # Build the model
        inputs = Input(shape=(X_train.shape[1],))
        x = Dense(128, activation='relu')(inputs)
        x = Dense(64, activation='relu')(x)
        outputs = Dense(7, activation='softmax')(x)
        model = Model(inputs, outputs)

        # Compile the model with focal loss
        model.compile(optimizer=Adam(),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics = [tf.keras.metrics.Precision(name='precision')])


        # Fit the model
        model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

        # Predict probabilities
        y_pred_proba = model.predict(X_test)

        # Adjust the threshold
        thresholds = np.arange(0.1, 0.9, 0.1)
        best_threshold = 0.5
        best_score = 0
        for threshold in thresholds:
            y_pred = np.argmax(y_pred_proba >= threshold, axis=1)
            score = precision_score(np.argmax(y_test, axis=1), y_pred, average='macro')
            if score > best_score:
                best_score = score
                best_threshold = threshold

        # Predict with the best threshold
        y_pred = np.argmax(y_pred_proba >= best_threshold, axis=1)

        score = precision_score(np.argmax(y_test, axis=1), y_pred, average='macro')
        scores.append(score)
        print(f"Fold score with threshold {best_threshold}: {score}")

        # Print classification report for this fold
        print(classification_report(np.argmax(y_test, axis=1), y_pred))

    print(f"Custom Logistic Regression Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating Logistic Regression: {e}")
    traceback.print_exc()

Epoch 1/20
3334/3334 [==============================] - 11s 3ms/step - loss: 0.0778 - precision: 0.8586
Epoch 2/20
3334/3334 [==============================] - 10s 3ms/step - loss: 0.0687 - precision: 0.8651
Epoch 3/20
3334/3334 [==============================] - 10s 3ms/step - loss: 0.0656 - precision: 0.8666
Epoch 4/20
3334/3334 [==============================] - 10s 3ms/step - loss: 0.0628 - precision: 0.8681
Epoch 5/20
3334/3334 [==============================] - 10s 3ms/step - loss: 0.0600 - precision: 0.8698
Epoch 6/20
3334/3334 [==============================] - 10s 3ms/step - loss: 0.0576 - precision: 0.8708
Epoch 7/20
3334/3334 [==============================] - 9s 3ms/step - loss: 0.0551 - precision: 0.8718
Epoch 8/20
3334/3334 [==============================] - 9s 3ms/step - loss: 0.0527 - precision: 0.8734
Epoch 9/20
3334/3334 [==============================] - 9s 3ms/step - loss: 0.0500 - precision: 0.8750
Epoch 10/20
3334/3334 [==============================] - 9s 3ms/ste

KeyboardInterrupt: 

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, precision_score, classification_report
import numpy as np
import traceback

# Apply SMOTE to increase data points for specified classes
smote = SMOTE(sampling_strategy={
    0: 7000,  # trivial
    1: 15000,  # enhancement
    2: 10000,  # minor
    4: 20000,  # major
    5: 5000,   # blocker
    6: 50000  # critical
}, random_state=42)

# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []

# Initialize the SVM classifier
clf = SVC(kernel='rbf', verbose=True)

# Scoring method
scorer = make_scorer(precision_score, average='macro')

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        clf.fit(X_train_res, y_train_res)
        y_pred = clf.predict(X_test)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"SVM Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating SVM: {e}")
    traceback.print_exc()


[LibSVM]

#Stacking

In [11]:
import numpy as np
import traceback
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, precision_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

# Define focal loss function for XGBoost
def focal_loss(alpha, gamma):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-7
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        fl = - alpha_t * np.power(1. - p_t, gamma) * np.log(p_t)
        return np.mean(fl)
    return focal_loss_fixed

# Apply SMOTE to increase data points for specified classes
smote = SMOTE(sampling_strategy={
    0: 7000,  # trivial
    1: 15000,  # enhancement
    2: 10000,  # minor
    4: 20000,  # major
    5: 5000,   # blocker
    6: 50000  # critical
}, random_state=42)

# Initialize classifiers
rf_clf = RandomForestClassifier(verbose=2, n_jobs=-1, n_estimators=200, criterion="entropy")
xgb_clf = XGBClassifier(objective='multi:softmax', eval_metric=focal_loss(alpha=0.25, gamma=2.0), use_label_encoder=False, n_jobs=-1, silent=0, n_estimators=200, reg_alpha=0.5, reg_lambda=1.5)

# Define the meta-learner
meta_clf = LogisticRegression(max_iter=1000, multi_class='multinomial', n_jobs=-1)





# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []

# Scoring method
scorer = make_scorer(precision_score, average='macro')

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Apply SMOTE only to the XGBoost training set
        X_train_res_xgb, y_train_res_xgb = smote.fit_resample(X_train, y_train)

        # Fit the Random Forest classifier on the original training set
        rf_clf.fit(X_train, y_train)

        # Fit the XGBoost classifier on the SMOTE-resampled training set
        xgb_clf.fit(X_train_res_xgb, y_train_res_xgb)

        # Generate predictions using the fitted classifiers
        rf_preds = rf_clf.predict(X_test)
        xgb_preds = xgb_clf.predict(X_test)

        # Stack predictions as new features for the meta-learner
        X_stack = np.column_stack((rf_preds, xgb_preds))

        # Fit the meta-learner
        meta_clf.fit(X_stack, y_test)

        # Generate final predictions using the stacking classifier
        y_pred = meta_clf.predict(X_stack)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"Stacking Classifier Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating Stacking Classifier: {e}")
    traceback.print_exc()


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=-1)]: Done 110 out of 200 | elapsed:   26.6s remaining:   21.8s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:   34.1s finished
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:09:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done 110 out of 200 | elapsed:    0.5s remaining:    0.4s
[Parallel(n_jobs=96)]: Done 200 out of 200 | elapsed:    0.6s finished
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision

Fold score: 0.32686157828720747
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       401
           1       0.67      0.00      0.01      1475
           2       0.00      0.00      0.00      1034
           3       0.86      0.97      0.91     41952
           4       0.00      0.00      0.00      2018
           5       0.00      0.00      0.00       234
           6       0.76      0.74      0.75      6219

    accuracy                           0.85     53333
   macro avg       0.33      0.24      0.24     53333
weighted avg       0.78      0.85      0.81     53333



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200building tree 15 of 200

building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

KeyboardInterrupt: 

#Custom Macro avp

In [21]:
import numpy as np
import traceback
from sklearn.metrics import precision_score, make_scorer, classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


smote = SMOTE(sampling_strategy={
    0: 7000,  # trivial
    1: 15000,  # enhancement
    2: 10000,  # minor
    4: 20000,  # major
    5: 5000,   # blocker
    6 : 50000  # critical
}, random_state=42)

# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []

# Initialize the Logistic Regression classifier
clf = LogisticRegression(max_iter=200, multi_class='multinomial', n_jobs=-1, )

# Scoring method
scorer = make_scorer(precision_score, average='macro')

# Standardize the data
scaler = StandardScaler()

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]



        # Standardize the features
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Fit the model
        clf.fit(X_train, y_train)

        # Predict probabilities
        y_pred_proba = clf.predict_proba(X_test)

        # Adjust the threshold
        thresholds = np.arange(0.1, 0.9, 0.1)
        best_threshold = 0.5
        best_score = 0
        for threshold in thresholds:
            y_pred = np.argmax(y_pred_proba >= threshold, axis=1)
            score = precision_score(y_test, y_pred, average='macro')
            if score > best_score:
                best_score = score
                best_threshold = threshold

        # Predict with the best threshold
        y_pred = np.argmax(y_pred_proba >= best_threshold, axis=1)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score with threshold {best_threshold}: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"Logistic Regression Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating Logistic Regression: {e}")
    traceback.print_exc()


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold score with threshold 0.6: 0.6330937416850102
              precision    recall  f1-score   support

           0       0.00      0.01      0.00       401
           1       0.55      0.01      0.02      1475
           2       1.00      0.00      0.00      1034
           3       0.87      0.96      0.91     41952
           4       0.50      0.00      0.00      2018
           5       0.67      0.01      0.02       234
           6       0.84      0.63      0.72      6219

    accuracy                           0.83     53333
   macro avg       0.63      0.23      0.24     53333
weighted avg       0.84      0.83      0.80     53333



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold score with threshold 0.7000000000000001: 0.5115832424692834
              precision    recall  f1-score   support

           0       0.00      0.06      0.01       402
           1       0.62      0.00      0.01      1476
           2       0.00      0.00      0.00      1034
           3       0.88      0.92      0.90     41951
           4       1.00      0.00      0.00      2017
           5       0.20      0.00      0.01       233
           6       0.88      0.55      0.67      6220

    accuracy                           0.79     53333
   macro avg       0.51      0.22      0.23     53333
weighted avg       0.85      0.79      0.78     53333

Fold score with threshold 0.30000000000000004: 0.48812637951859345
              precision    recall  f1-score   support

           0       0.14      0.01      0.01       401
           1       0.32      0.08      0.12      1475
           2       0.38      0.02      0.03      1034
           3       0.85      0.98      0.91     41951


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Grid Search

In [ ]:

# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []

# Define parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'n_estimators': [100, 200, 300, 400],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.5, 1.0],  # L1 regularization term
    'reg_lambda': [0.5, 1.0, 1.5, 2.0]  # L2 regularization term
}

# Initialize the XGBoost classifier
xgb_clf = XGBClassifier(objective='multi:softmax', use_label_encoder=False, n_jobs=-1, silent=0)

# Initialize RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_grid, scoring='precision_macro', cv=3, verbose=2, n_jobs=-1, n_iter=50, random_state=42)

# Scoring method
scorer = make_scorer(precision_score, average='macro')

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Perform hyperparameter tuning
        random_search.fit(X_train, y_train)
        best_model = random_search.best_estimator_

        y_pred = best_model.predict(X_test)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"XGBoost Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating XGBoost: {e}")
    traceback.print_exc()

In [ ]:
import numpy as np
import traceback
from sklearn.metrics import precision_score, classification_report
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

# Define a function to evaluate a set of hyperparameters
def evaluate_params(params, X_train, y_train, X_test, y_test):
    clf = XGBClassifier(**params,eval_metric=focal_loss(alpha=0.25, gamma=2.0) ,use_label_encoder=False, n_jobs=-1, objective='multi:softmax', silent=0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = precision_score(y_test, y_pred, average='macro')
    return score, y_pred

# Define hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [7, 10, ],
    'n_estimators': [100, 200],
    'subsample': [0.6, 1.0],
    'colsample_bytree': [0.6, 1.0],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0.5, 1.0],  # L1 regularization term
    'reg_lambda': [0.5, 1.0, 2.0]  # L2 regularization term
}

# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []
best_score = -1
best_params = None

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        for lr in param_grid['learning_rate']:
            for md in param_grid['max_depth']:
                for ne in param_grid['n_estimators']:
                    for ss in param_grid['subsample']:
                        for cs in param_grid['colsample_bytree']:
                            for mcw in param_grid['min_child_weight']:
                                for g in param_grid['gamma']:
                                    for ra in param_grid['reg_alpha']:
                                        for rl in param_grid['reg_lambda']:
                                            params = {
                                                'learning_rate': lr,
                                                'max_depth': md,
                                                'n_estimators': ne,
                                                'subsample': ss,
                                                'colsample_bytree': cs,
                                                'min_child_weight': mcw,
                                                'gamma': g,
                                                'reg_alpha': ra,
                                                'reg_lambda': rl
                                            }
                                            score, y_pred = evaluate_params(params, X_train, y_train, X_test, y_test)
                                            if score > best_score:
                                                best_score = score
                                                best_params = params
                                            scores.append(score)
                                            print(f"Evaluating params: {params} -> Score: {score}")
                                            # Print classification report for this fold
                                            print(classification_report(y_test, y_pred))

    print(f"Best parameters found: {best_params}")
    print(f"Best Macro Precision: {best_score}")
    print(f"Average Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating XGBoost: {e}")
    traceback.print_exc()


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:06:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

Evaluating params: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0, 'reg_alpha': 0.5, 'reg_lambda': 0.5} -> Score: 0.6138271883395146
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       401
           1       1.00      0.00      0.00      1475
           2       0.00      0.00      0.00      1034
           3       0.84      0.99      0.91     41952
           4       0.62      0.00      0.01      2018
           5       1.00      0.00      0.01       234
           6       0.84      0.51      0.63      6219

    accuracy                           0.84     53333
   macro avg       0.61      0.22      0.22     53333
weighted avg       0.81      0.84      0.79     53333



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:07:16] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

Evaluating params: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0, 'reg_alpha': 0.5, 'reg_lambda': 1.0} -> Score: 0.6091244550885778
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       401
           1       1.00      0.00      0.00      1475
           2       0.00      0.00      0.00      1034
           3       0.84      0.99      0.91     41952
           4       0.58      0.00      0.01      2018
           5       1.00      0.00      0.01       234
           6       0.84      0.51      0.63      6219

    accuracy                           0.84     53333
   macro avg       0.61      0.22      0.22     53333
weighted avg       0.81      0.84      0.79     53333



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:08:13] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

Evaluating params: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0, 'reg_alpha': 0.5, 'reg_lambda': 2.0} -> Score: 0.6074015303004726
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       401
           1       1.00      0.00      0.00      1475
           2       0.00      0.00      0.00      1034
           3       0.84      0.99      0.91     41952
           4       0.57      0.00      0.01      2018
           5       1.00      0.00      0.01       234
           6       0.84      0.51      0.63      6219

    accuracy                           0.84     53333
   macro avg       0.61      0.22      0.22     53333
weighted avg       0.81      0.84      0.79     53333



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:09:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

Evaluating params: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.6, 'colsample_bytree': 0.6, 'min_child_weight': 1, 'gamma': 0, 'reg_alpha': 1.0, 'reg_lambda': 0.5} -> Score: 0.6177257883233433
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       401
           1       1.00      0.00      0.00      1475
           2       0.00      0.00      0.00      1034
           3       0.84      0.99      0.91     41952
           4       0.64      0.00      0.01      2018
           5       1.00      0.00      0.01       234
           6       0.84      0.51      0.63      6219

    accuracy                           0.84     53333
   macro avg       0.62      0.22      0.22     53333
weighted avg       0.81      0.84      0.79     53333



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:10:05] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


# XGBoost

In [16]:

# Define focal loss function
def focal_loss(alpha, gamma):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1e-7
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
        alpha_t = y_true * alpha + (1 - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        fl = - alpha_t * np.power(1. - p_t, gamma) * np.log(p_t)
        return np.mean(fl)
    return focal_loss_fixed


# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []

# Initialize the XGBoost classifier
clf = XGBClassifier(objective='multi:softmax', eval_metric=focal_loss(alpha=1, gamma=2.5), use_label_encoder=False,n_jobs=-1, silent=0, n_estimators=200, reg_alpha=0.5, reg_lambda=1.5)

# Scoring method
scorer = make_scorer(precision_score, average='macro')

try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train_res, y_train_res =X_train, y_train

        clf.fit(X_train_res, y_train_res, )
        y_pred = clf.predict(X_test)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"XGBoost Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating XGBoost: {e}")
    traceback.print_exc()

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:33:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold score: 0.7607475970018527
              precision    recall  f1-score   support

           0       0.50      0.00      0.01       401
           1       0.76      0.02      0.03      1475
           2       0.67      0.01      0.02      1034
           3       0.86      0.98      0.92     41952
           4       0.72      0.05      0.10      2018
           5       1.00      0.01      0.03       234
           6       0.83      0.68      0.75      6219

    accuracy                           0.85     53333
   macro avg       0.76      0.25      0.26     53333
weighted avg       0.84      0.85      0.81     53333



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:35:28] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold score: 0.766849760419167
              precision    recall  f1-score   support

           0       0.60      0.01      0.01       402
           1       0.76      0.02      0.03      1476
           2       0.60      0.01      0.02      1034
           3       0.86      0.98      0.92     41951
           4       0.71      0.05      0.09      2017
           5       1.00      0.02      0.03       233
           6       0.84      0.68      0.75      6220

    accuracy                           0.86     53333
   macro avg       0.77      0.25      0.27     53333
weighted avg       0.84      0.86      0.81     53333



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:37:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fold score: 0.7501596586571353
              precision    recall  f1-score   support

           0       0.67      0.00      0.01       401
           1       0.73      0.02      0.04      1475
           2       0.86      0.01      0.01      1034
           3       0.86      0.98      0.92     41951
           4       0.67      0.04      0.08      2018
           5       0.62      0.02      0.04       234
           6       0.84      0.68      0.75      6219

    accuracy                           0.86     53332
   macro avg       0.75      0.25      0.26     53332
weighted avg       0.84      0.86      0.81     53332

XGBoost Macro Precision: 0.7592523386927184


# Random Forest

In [12]:
# Apply SMOTE to increase data points for specified classes

smote = SMOTE(sampling_strategy={
    0: 7000,  # trivial
    1: 15000,  # enhancement
    2: 10000,  # minor
    4: 20000,  # major
    5: 5000,   # blocker
    6 : 50000  # critical
}, random_state=42)



# Apply Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = []



# Initialize the Random Forest classifier
clf = RandomForestClassifier(verbose=2, n_jobs=-1, n_estimators=300, criterion="gini",max_depth=None, min_samples_leaf=2,min_samples_split=10,)

# Scoring method
scorer = make_scorer(precision_score, average='macro')



try:
    for train_index, test_index in skf.split(X_encoded, y):
        X_train, X_test = X_encoded[train_index], X_encoded[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

        clf.fit(X_train_res, y_train_res)
        y_pred = clf.predict(X_test)

        score = precision_score(y_test, y_pred, average='macro')
        scores.append(score)
        print(f"Fold score: {score}")

        # Print classification report for this fold
        print(classification_report(y_test, y_pred))

    print(f"Random Forest Macro Precision: {np.mean(scores)}")

except Exception as e:
    print(f"Error evaluating Random Forest: {e}")
    traceback.print_exc()


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300building tree 41 of 300

building tree 42 of 300
b

[Parallel(n_jobs=-1)]: Done 260 out of 300 | elapsed:   53.4s remaining:    8.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.0min finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.
[Parallel(n_jobs=96)]: Done 260 out of 300 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=96)]: Done 300 out of 300 | elapsed:    0.6s finished


Fold score: 0.6861904716627579
              precision    recall  f1-score   support

           0       0.50      0.00      0.01       401
           1       0.56      0.01      0.03      1475
           2       0.38      0.00      0.01      1034
           3       0.86      0.98      0.91     41952
           4       0.73      0.04      0.07      2018
           5       1.00      0.01      0.03       234
           6       0.78      0.68      0.73      6219

    accuracy                           0.85     53333
   macro avg       0.69      0.25      0.25     53333
weighted avg       0.82      0.85      0.81     53333



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 96 concurrent workers.


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300building tree 24 of 300

building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300
building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
b

[Parallel(n_jobs=-1)]: Done 260 out of 300 | elapsed:   51.8s remaining:    8.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   58.3s finished
[Parallel(n_jobs=96)]: Using backend ThreadingBackend with 96 concurrent workers.


Fold score: 0.7623242225775517
              precision    recall  f1-score   support

           0       0.75      0.01      0.01       401
           1       0.63      0.01      0.03      1475
           2       0.88      0.01      0.01      1034
           3       0.86      0.98      0.91     41951
           4       0.80      0.04      0.07      2018
           5       0.64      0.03      0.06       234
           6       0.79      0.68      0.73      6219

    accuracy                           0.85     53332
   macro avg       0.76      0.25      0.26     53332
weighted avg       0.84      0.85      0.81     53332

Random Forest Macro Precision: 0.7175851924713378


[Parallel(n_jobs=96)]: Done 260 out of 300 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=96)]: Done 300 out of 300 | elapsed:    0.6s finished


In [18]:

# Train on the entire dataset
try:
    X_train, y_train = X_encoded, y
    clf.fit(X_train, y_train)
except Exception as e:
    print(f"Error training Random Forest on the entire dataset: {e}")
    traceback.print_exc()





/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [20:44:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


In [19]:


# Make predictions on the test dataset
test_predictions = clf.predict(X_test_encoded)



# Inverse mapping
inverse_severity_mapping = {v: k for k, v in severity_mapping.items()}
test_data['severity_mapped'] = test_predictions
test_data['severity'] = test_data['severity_mapped'].map(inverse_severity_mapping)

# Save the test predictions
test_data.rename(columns={"bug id": "bug_id"}, inplace=True)
output_data = test_data[['bug_id', 'severity']]
output_file_path = 'rf_bert_predictions_11.csv'
output_data.to_csv(output_file_path, index=False)